In [85]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import pickle


In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
df = pd.read_excel('/content/drive/MyDrive/salesdaily.xlsx')

In [88]:
df['datum'] = pd.to_datetime(df['datum'])

In [89]:
def quantity(t):
  fig = px.line(df, x='datum', y='M01AB', title='Quantity of M01AB over time')
  return fig

In [90]:
df_m01ab = df[['M01AB','Year','Month']]
df_m01ab

,M01AB,Year,Month
0,0.00,2014,1
1,8.00,2014,1
2,2.00,2014,1
3,4.00,2014,1
4,5.00,2014,1
...,...,...,...
2101,7.34,2019,10
2102,3.84,2019,10
2103,4.00,2019,10
2104,7.34,2019,10


In [91]:
df_m01ab = df_m01ab.groupby(['Year', 'Month']).sum().reset_index()
df_m01ab

,Year,Month,M01AB
0,2014,1,127.69
1,2014,2,133.32
2,2014,3,137.44
3,2014,4,113.10
4,2014,5,101.79
...,...,...,...
65,2019,6,151.54
66,2019,7,181.00
67,2019,8,181.91
68,2019,9,161.07


In [92]:
fig = px.bar(df_m01ab, x='Month', y='M01AB', color='Year', barmode='group')

# Update the axis labels and title
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='M01AB',
    title='M01AB vs Month and Year'
)

# Show the plot
fig.show()

In [93]:
df_m01ab['Date'] = pd.to_datetime(df_m01ab[['Year', 'Month']].assign(day=1))

# Plot M01AB vs date
fig = px.line(df_m01ab, x='Date', y='M01AB', title='M01AB vs Date')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='M01AB')

# Show the plot
fig.show()


In [94]:
df['day'] = df['datum'].dt.day
df

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name,day
0,2014-01-02,0.00,3.670,3.40,32.40,7.0,0.0,0.0,2.00,2014,1,248,Thursday,2
1,2014-01-03,8.00,4.000,4.40,50.60,16.0,0.0,20.0,4.00,2014,1,276,Friday,3
2,2014-01-04,2.00,1.000,6.50,61.85,10.0,0.0,9.0,1.00,2014,1,276,Saturday,4
3,2014-01-05,4.00,3.000,7.00,41.10,8.0,0.0,3.0,0.00,2014,1,276,Sunday,5
4,2014-01-06,5.00,1.000,4.50,21.70,16.0,2.0,6.0,2.00,2014,1,276,Monday,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,2019-10-04,7.34,5.683,2.25,22.45,13.0,0.0,1.0,1.00,2019,10,276,Friday,4
2102,2019-10-05,3.84,5.010,6.00,25.40,7.0,0.0,0.0,0.33,2019,10,276,Saturday,5
2103,2019-10-06,4.00,11.690,2.00,34.60,6.0,0.0,5.0,4.20,2019,10,276,Sunday,6
2104,2019-10-07,7.34,4.507,3.00,50.80,6.0,0.0,10.0,1.00,2019,10,276,Monday,7


In [95]:
melted_df = pd.melt(df, id_vars=['Year', 'Month'], value_vars=['M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B', 'N05C', 'R03', 'R06'],
                    var_name='Category', value_name='Consumption')

# Group the data by Category and Month and calculate the total consumption
grouped_df = melted_df.groupby(['Category', 'Month']).sum().reset_index()

# Create the bar chart
fig = px.bar(grouped_df, x='Month', y='Consumption', color='Category', barmode='group')

# Customize the layout
fig.update_layout(
    title='Total Monthly Consumption of Each Category',
    xaxis_title='Month',
    yaxis_title='Consumption',
    legend_title='Category',
)

# Show the chart
fig.show()


In [96]:
df[df['datum'].dt.month == 1]['M01AB'].sum()


929.1320833339998

In [97]:
df_m01ab.groupby('Year')['M01AB'].sum().reset_index()

,Year,M01AB
0,2014,1447.215000
1,2015,1895.620000
2,2016,2107.285000
3,2017,1846.617083
4,2018,1786.930000
5,2019,1517.270000


In [98]:
dfs = df_m01ab.groupby('Year')['M01AB'].sum().reset_index()

# Create the bar chart
fig = px.bar(dfs, x='Year', y='M01AB', color='Year')

# Customize the layout
fig.update_layout(
    title='Total Yearly Consumption of M01AB',
    xaxis_title='Year',
    yaxis_title='Consumption',
    showlegend=False
)

# Show the chart
fig.show()


In [99]:
df_new = df.melt(id_vars=['datum', 'Year', 'Month', 'Hour', 'Weekday Name', 'day'],
             var_name='Drug',
             value_name='Quantity')



In [100]:

le = LabelEncoder()
df_new['Weekday Name'] = le.fit_transform(df_new['Weekday Name'])
df_new['Drug'] = le.fit_transform(df_new['Drug'])
df_new

,datum,Year,Month,Hour,Weekday Name,day,Drug,Quantity
0,2014-01-02,2014,1,248,4,2,0,0.00
1,2014-01-03,2014,1,276,0,3,0,8.00
2,2014-01-04,2014,1,276,2,4,0,2.00
3,2014-01-05,2014,1,276,3,5,0,4.00
4,2014-01-06,2014,1,276,1,6,0,5.00
...,...,...,...,...,...,...,...,...
16843,2019-10-04,2019,10,276,0,4,7,1.00
16844,2019-10-05,2019,10,276,2,5,7,0.33
16845,2019-10-06,2019,10,276,3,6,7,4.20
16846,2019-10-07,2019,10,276,1,7,7,1.00


In [101]:
df_new.set_index('datum')

,Year,Month,Hour,Weekday Name,day,Drug,Quantity
datum,,,,,,,
2014-01-02,2014,1,248,4,2,0,0.00
2014-01-03,2014,1,276,0,3,0,8.00
2014-01-04,2014,1,276,2,4,0,2.00
2014-01-05,2014,1,276,3,5,0,4.00
2014-01-06,2014,1,276,1,6,0,5.00
...,...,...,...,...,...,...,...
2019-10-04,2019,10,276,0,4,7,1.00
2019-10-05,2019,10,276,2,5,7,0.33
2019-10-06,2019,10,276,3,6,7,4.20


In [102]:
train = df_new[df_new['Year'] < 2019]
test = df_new[df_new['Year'] >= 2019]

In [103]:
X_train = train.drop(['Hour','Quantity','datum'],axis = 1)
y_train = train['Quantity']
from sklearnex import patch_sklearn
patch_sklearn()
reg = xgb.XGBRegressor(n_estimators = 1000,early_stopping_rounds = 50, learning_rate = 0.005)
reg.fit(X_train,y_train,eval_set = [(X_train,y_train),(X_test,y_test)],verbose = 10)

[0]	validation_0-rmse:10.89224	validation_1-rmse:10.62382
[10]	validation_0-rmse:10.51170	validation_1-rmse:10.28341
[20]	validation_0-rmse:10.15388	validation_1-rmse:9.97119
[30]	validation_0-rmse:9.81878	validation_1-rmse:9.67917
[40]	validation_0-rmse:9.50530	validation_1-rmse:9.40628
[50]	validation_0-rmse:9.21236	validation_1-rmse:9.15171
[60]	validation_0-rmse:8.93704	validation_1-rmse:8.91287
[70]	validation_0-rmse:8.67996	validation_1-rmse:8.69032
[80]	validation_0-rmse:8.44044	validation_1-rmse:8.47823
[90]	validation_0-rmse:8.21709	validation_1-rmse:8.28044
[100]	validation_0-rmse:8.00798	validation_1-rmse:8.10052


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


[110]	validation_0-rmse:7.81350	validation_1-rmse:7.93365
[120]	validation_0-rmse:7.63286	validation_1-rmse:7.77875
[130]	validation_0-rmse:7.46479	validation_1-rmse:7.63540
[140]	validation_0-rmse:7.30917	validation_1-rmse:7.50354
[150]	validation_0-rmse:7.16393	validation_1-rmse:7.38461
[160]	validation_0-rmse:7.02930	validation_1-rmse:7.27294
[170]	validation_0-rmse:6.90458	validation_1-rmse:7.17032
[180]	validation_0-rmse:6.78911	validation_1-rmse:7.07421
[190]	validation_0-rmse:6.68239	validation_1-rmse:6.98575
[200]	validation_0-rmse:6.58379	validation_1-rmse:6.90438
[210]	validation_0-rmse:6.49263	validation_1-rmse:6.82895
[220]	validation_0-rmse:6.40744	validation_1-rmse:6.75954
[230]	validation_0-rmse:6.32823	validation_1-rmse:6.69475
[240]	validation_0-rmse:6.25464	validation_1-rmse:6.63535
[250]	validation_0-rmse:6.18636	validation_1-rmse:6.58353
[260]	validation_0-rmse:6.12311	validation_1-rmse:6.53609
[270]	validation_0-rmse:6.06451	validation_1-rmse:6.49235
[280]	validati

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.005, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [104]:
param_grid = {
    'n_estimators': [100, 500, 1000],    # Number of trees in the forest
    'learning_rate': [0.01, 0.1, 0.2],   # Learning rate
    'max_depth': [3, 5, 7],               # Maximum depth of each tree
    'subsample': [0.8, 1.0],              # Subsample ratio of the training instances
    'colsample_bytree': [0.8, 1.0]        # Subsample ratio of columns when constructing each tree
}
# Create the XGBoost regressor

In [105]:
xgb = xgb.XGBRegressor(random_state=42)

In [106]:
grid_search = GridSearchCV(xgb, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 500, 1000],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error')

In [107]:
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Mean Squared Error:", -grid_search.best_score_)

# Evaluate the model on the test set using the best hyperparameters
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Test Root Mean Squared Error:", rmse)

Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500, 'subsample': 0.8}
Best Mean Squared Error: 71.75971527436408
Test Root Mean Squared Error: 5.881822312355689


In [108]:
test['Quantity Predictions'] = y_pred
test

<ipython-input-108-7a9dbaac5b85>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,datum,Year,Month,Hour,Weekday Name,day,Drug,Quantity,Quantity Predictions
1825,2019-01-01,2019,1,276,5,1,0,0.00,-1.069816
1826,2019-01-02,2019,1,276,6,2,0,5.33,4.118169
1827,2019-01-03,2019,1,276,4,3,0,4.33,5.685684
1828,2019-01-04,2019,1,276,0,4,0,7.00,5.949047
1829,2019-01-05,2019,1,276,2,5,0,8.01,5.677755
...,...,...,...,...,...,...,...,...,...
16843,2019-10-04,2019,10,276,0,4,7,1.00,3.623539
16844,2019-10-05,2019,10,276,2,5,7,0.33,3.390931
16845,2019-10-06,2019,10,276,3,6,7,4.20,3.475652
16846,2019-10-07,2019,10,276,1,7,7,1.00,2.875281


In [109]:
import pickle
# save the model to disk
filename = 'pharma_model.sav'

pickle.dump(best_xgb, open(filename, 'wb'))
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [110]:
def predict_sales(start_date,end_date,drug): # dates selected from celander and category(int) from options
    # Generate a range of dates
    dates = pd.date_range(start=start_date, end=end_date, freq='D')

    # Create the DataFrame with dates as the index
    df_test = pd.DataFrame(index=dates)
    df_test['Year'] = df_test.index.year
    df_test['Month'] = df_test.index.month
    df_test['Weekday Name'] = df_test.index.weekday
    df_test['day'] = df_test.index.day
    df_test['Drug'] = drug
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df_test['Weekday Name'] = le.fit_transform(df_test['Weekday Name'])
    df_test['predicted_quantity'] = loaded_model.predict(df_test)
    return df_test

In [111]:
predict_sales(start_date = '2023-10-01',end_date = '2023-10-31',drug = 3)

,Year,Month,Weekday Name,day,Drug,predicted_quantity
2023-10-01,2023,10,6,1,3,34.535648
2023-10-02,2023,10,0,2,3,36.836823
2023-10-03,2023,10,1,3,3,37.718216
2023-10-04,2023,10,2,4,3,40.204460
2023-10-05,2023,10,3,5,3,40.500534
2023-10-06,2023,10,4,6,3,36.603947
2023-10-07,2023,10,5,7,3,36.134537
2023-10-08,2023,10,6,8,3,35.998726
2023-10-09,2023,10,0,9,3,36.823532
2023-10-10,2023,10,1,10,3,38.729351
